___

# Atividade - Regressão Linear Múltipla

## Aula 23
___

<div id="indice"></div>

## Índice

- [Consumo de combustível em milhas por galão](#dataset)
    - [Passo 1: Ajustando o modelo](#passo1)
    - [Passo 2: Verificando os resultados dos testes de hipótese](#passo2)
    - [Passo 3: Visualizando o modelo e os dados (com duas variáveis explicativas)](#passo3)
- [Atividade](#atividade)
    - [Exercício 1](#ex1)
    - [Exercício 2](#ex2)
    - [Exercício 3](#ex3)
    - [Exercício 4](#ex4)
    - [Exercício 5](#ex5)
- [Referências](#referencias)

___


<div id="dataset"></div>

# Consumo de combustível em milhas por galão

A atividade a realizar está no final, mas estude todo o *notebook*

In [ ]:
%matplotlib notebook
%reset -f


import pandas as pd
import numpy as np
from scipy.stats import norm, probplot
import statsmodels.api as sm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


Vamos fazer uma análise exploratória sobre um dataset sobre carros.

Fonte: [Auto MPG dataset](http://archive.ics.uci.edu/ml/datasets/Auto+MPG)

<div id="regress"></div>

In [ ]:
### Função utilitária para fazer a regressão com constante adicionada
def regress(X,Y):
    '''
    X: coluna(s) do DataFrame que serão utilizadas como variável(is) preditoras/explicativas (independente(s))
    Y: coluna do DataFrame que será utilizada como variável resposta (dependente)
    '''
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

In [ ]:
data = pd.read_csv("auto_mpg_sub.csv")

**Dicionário de dados:**


Variável | Descrição
:---:|---:
mpg | consumo em miles per gallon
cylinders | número de cilindros do motor do carro
displacement | deslocamento dos cilindros em polegadas cúbicas
horsepower | potência do veículo (HP)
weight | peso do veículo (libras)
acceleration | aceleração do veículo
model year | ano do modelo
origin | origem - veja abaixo
car name | nome do carro

A variável *origin* identifica o país de origem de um veículo

Valor | Origem
:---:|:---:
1 | Estados Unidos da América
2 | Europa
3 | Japão

In [ ]:
data.columns

**Vamos trabalhar apenas com as variáveis quantitativas desse dataset**

In [ ]:
df = data[['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year']]

In [ ]:
df.describe()

Verificando valores nulos.

In [ ]:
np.sum(df.isnull())

___

<div id="passo1"></div>

### Passo 1: Ajustando o modelo

Ajuste um modelo considerando Consumo como resposta (dependente) e as demais variáveis da base de dados como explicativas!

Faça a separação das variáveis resposta e preditoras (explicativas):

In [ ]:
X = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model year']]
Y = df['mpg']

Faça uso da função [`regress`](#regress) (que está neste *notebook* ) para fazer o ajuste:

In [ ]:
results = regress(X,Y)
results.summary()

___

<div id="passo2"></div>

### Passo 2: Verificando os resultados dos testes de hipótese

O que os valores $P > |t|$ dizem a respeito das variáveis preditoras usadas?


**R.:**

A coluna $P > |t|$ obtida do comando `results.summary()` refere-se ao valor *p* de cada variável explicativa utilizada para ajudar a compreender o comportamento da variável dependente `mpg` no caso do exemplo.

No caso da regressão múltipla obtida no exemplo, das seis variáveis explicativas, apenas duas se mostram relevantes para explicar a variável `mpg` pois possui valor *p* menor do que 10% (maior nível de significância usualmente adotado na prática).
Entretanto, como há muitas variáveis explicativas, o correto é utilizar algum método de seleção de variáveis. Em modelo de regressão linear, quando há suposição de erros normais, pode-se utilizar o método Stepwise. Vide alguns comentários de como fazê-lo no `Python´.

https://stackoverflow.com/questions/15433372/stepwise-regression-in-python

https://stackoverflow.com/questions/48071753/selecting-the-best-combination-of-variables-for-regression-model-based-on-reg-sc

___

<div id="passo3"></div>

### Passo 3: Visualizando o modelo e os dados (com duas variáveis explicativas)

Observe como plotar os resultados de uma regressão múltipla quando há duas variáveis explicativas:

In [ ]:
# Obtém a regressão
X3 = df[['cylinders',
       'acceleration']]
results3 = regress(X3, Y)
results3.summary()

In [ ]:
# Recupera os parâmetros calculados
c = results3.params["const"]
w = results3.params["cylinders"]
y = results3.params["acceleration"]

In [ ]:
# Define uma função que aplica os parâmetros
def f(cylinders, acc):
    return c+w*cylinders + y*acc

In [ ]:
# Gera 100 pontos entre os menores e maiores valores de cada variável 
npoints = 100
cylinders = np.linspace(X3["cylinders"].min(), X3["cylinders"].max(), npoints)
acc = np.linspace(X3["acceleration"].min(), X3["acceleration"].max(), npoints)


In [ ]:
# Constrói uma grade de 100x100 pontos 
cylinders_xx, acc_yy = np.meshgrid(cylinders, acc)

In [ ]:
# Aplica a função em cada ponto da grade
z = f(cylinders_xx, acc_yy)

In [ ]:
# Plota os pontos e o plano em 3D
plt3d = plt.figure().gca(projection="3d")
plt3d.plot_surface(cylinders_xx, acc_yy, z)
plt3d.scatter(X3["cylinders"],X3["acceleration"], Y , color="r");

___

<div id="atividade"></div>

## Atividade

Parece que as variáveis `cylinders` e `acceleration` não são as melhores escolhas para explicar a economia de combustível.

Encontre $2$ variáveis melhores, gere o modelo via `OLS` e realize o plot 3D. Esse objetivo será atingido ao completarmos os exercícios a seguir.

<div id="ex1"></div>

### EXERCÍCIO 1

Para encontrar as duas melhores variáveis que explicam a variável `mpg`, vamos aplicar o método *Backward Stepwise*. Esse método consiste em começar utilizando todas as variáveis disponíveis na regressão e remover as menos úteis para a explicação, de uma em uma, até sobrarem apenas duas. Para iniciar esse processo, ajuste um modelo linear com todas as variáveis disponíveis. **Dica**: isso já foi feito anteriormente nesta atividade e armazenado em uma variável. Você pode utilizar o mesmo resultado aqui.

In [ ]:
# ESCREVA SEU CÓDIGO AQUI

<div id="ex2"></div>

### EXERCÍCIO 2

Remova a variável com maior valor-p desde que não significante, ou seja, desde que seja maior do que o nível de significância (nesse caso, 10%)! No caso, verá que será removida a variável `horsepower`. Faça um novo ajuste de regressão sem essa variável! 

In [ ]:
# ESCREVA SEU CÓDIGO AQUI

<div id="ex3"></div>

### EXERCÍCIO 3

Do novo ajuste, verá que a variável `cylinders` possui maior valor-p > 10%. Faça novo ajuste sem essa variável. E assim sucessivamente até que todas as variáveis no modelo final tenham valor-p abaixo do nível de significância adotado (10%).

In [ ]:
# ESCREVA SEU CÓDIGO AQUI

<div id="ex4"></div>

### EXERCÍCIO 4

Na prática, em cada passo para remoção de uma variável é necessário avaliar a validade das suposições do modelo. Por questão de tempo, nesta atividade vamos fazer a análise detalhada das suposições do modelo apenas com o modelo final.

Utilizando os resultados da regressão com duas variáveis (modelo final), discuta se as suposições do modelo.

In [ ]:
# ESCREVA SUA RESPOSTA AQUI

<div id="ex5"></div>

### EXERCÍCIO 5

Faça o plot 3D com as duas variáveis relevantes para explicar a variável `mpg`. Note que podemos fazer isso apenas porque ambas possuem valor *p* < 10%.

In [ ]:
# ESCREVA SEU CÓDIGO AQUI

**OBSERVAÇÃO**

Note que se o modelo final ficasse com mais do que duas variáveis preditoras com valor *p* < 10%, então um gráfico 3D não seria adequado e nem informativo para conseguir avaliar o modelo ajustado sob os pontos!!

# Referências

1. Montogmery e Runger. Capítulo 12 - Regressão linear Múltipla
1. Magalhães e Lima, seção 9.5. Regressão Linear Simples
1. http://www.statsmodels.org/dev/diagnostic.html
1. [http://connor-johnson.com/2014/02/18/linear-regression-with-python/](http://connor-johnson.com/2014/02/18/linear-regression-with-python/)
